In [53]:
import gym
import numpy as np

import time

In [54]:
import sys
import os

sys.path.append(os.path.abspath('../'))

del sys, os

In [55]:
import matplotlib.pyplot as plt

In [56]:
# LaTeX rendering in graphs
from distutils.spawn import find_executable
if find_executable('latex'):
    plt.rc('text', usetex=True)

plt.rc('font', family='serif')

# High resolution graphs
%config InlineBackend.figure_format = 'retina'

In [57]:
import torch

In [58]:
%reload_ext autoreload
%autoreload 2

In [59]:
import models.rnn as rnns
import models.mlp as mlps
import models.linear as linears
import control.agents as agents
import control.environments as env

In [60]:
from utils.notifications import Slack

In [61]:
import copy

# Setup

In [62]:
env_name = 'Taxi-v2'
#env_name = 'Breakout-ram-v0'

In [63]:
environment = env.Environment(
    environment=gym.make(env_name), 
    agent=None,
    verbose=True,
    max_steps=200,
    capacity=100,
    representation_method='one_hot_encoding'
)

In [64]:
model = linears.Linear(
    input_dimension=environment.get_input_dimension(), 
    n_actions=environment.n_actions,
    bias=False
)

#optimiser = torch.optim.Adam(model.parameters(), lr=.5)
optimiser = torch.optim.SGD(model.parameters(), lr=10., momentum=0)

agent = agents.DQNAgent(
    model,
    optimiser, 
    gamma=1., 
    temperature=10, 
    algorithm='sarsa', 
    n_actions=environment.n_actions,
    terminal_state=environment.max_obs,
    use_double_learning=False
)

environment.agent = agent

In [13]:
print(environment.agent.q(environment.state_representation(479)))
print()
#model.load_state_dict(torch.load('../saved/taxi/mlp/state_dict.pth'))
model.load_state_dict(torch.load('../../../../Downloads/state_dict.pth'))

agent.commit()
print(environment.agent.q(environment.state_representation(1)))
print(environment.agent.q(environment.state_representation(66)))
print(environment.agent.q(environment.state_representation(479)))
print(environment.agent.q(environment.state_representation(499)))

[-0.01093328  0.02178707  0.01731317  0.01599102  0.00051596 -0.04388257]



RuntimeError: Error(s) in loading state_dict for Linear:
	size mismatch for layer.weight: copying a param with shape torch.Size([6, 500]) from checkpoint, the shape in current model is torch.Size([6, 501]).

# Experiment

## Boltzmann

In [66]:
q_estimation = []
returns = []

iterator = environment.tqdm(range(10), ascii=True, ncols=100)

with iterator as it:
    for _ in it:

        environment.agent.commit()
        returns.append(environment.exploration_segment(100))

        for _ in range(len(environment.replay_memory) // 100):
            environment.batch(100)


 10%|######4                                                         | 1/10 [00:02<00:25,  2.88s/it]

[0. 0. 0. 0. 0. 0.]
[ 9.97181988 10.94699955 11.67240238 10.2612505   6.33517933  2.33609772]


 20%|############8                                                   | 2/10 [00:06<00:23,  2.96s/it]

[0. 0. 0. 0. 0. 0.]
[ 4.28665161  3.4852581   3.58703041  3.73958969 -2.68236351 -4.64079666]


 30%|###################2                                            | 3/10 [00:08<00:20,  2.94s/it]

...

[0. 0. 0. 0. 0. 0.]
[ -3.93337059  -3.49136353  -3.79053497  -2.50132561  -9.0220108
 -11.30794525]


 90%|#########################################################6      | 9/10 [00:26<00:02,  2.88s/it]

[0. 0. 0. 0. 0. 0.]
[ -3.05098534  -2.69287491  -1.05437469  -2.71089363 -10.01164436
 -11.14951134]


100%|###############################################################| 10/10 [00:29<00:00,  3.01s/it]

[0. 0. 0. 0. 0. 0.]
[ -3.28458023  -5.95121956  -5.25034332  -5.53855324 -11.45821762
  -9.93468857]


In [ ]:
environment.reset()

done = False
full_return = 0.

counter = 0
while not done and counter < environment.max_steps:
    
    s, reward, done, i = environment.step(environment.action)

    p, q = environment.boltzmann(s, return_q=True)
    a = environment.sample_action(p)
    
    environment.state, environment.action = s, a

    full_return += reward
    counter += 1
    
    print(environment.agent.q(s))
    print(environment.action, np.argmax(environment.state))


In [26]:
n_episodes = 3
agent.temperature = 0.1

plt.figure()

for i in range(n_episodes):

    full_return, counter, observations = environment.evaluation_episode(render=False,return_observations=True)
    
    q = []
    
    observation_old = None
    q_old = None
    
    for observation in observations:
        
        observation_new = environment.state_representation(observation)
        q_new = environment.agent.q(environment.state_representation(observation))
        
        if observation_old is not None:
            same_obs = np.array_equal(observation_old, observation_new)
            same_q = np.array_equal(q_old, q_new)
            if not same_obs and not same_q:
                print("great")
        
        observation_old = observation_new
        q_old = q_new
        
        q.append(q_new)
        
    q = np.asarray(q)
    print(np.std(q, axis=0))
    print(q[:,0])
    plt.plot(q[:,0], label='0')
    break
    #plt.plot(q[:,1], label='1')
    #plt.plot(q[:,2], label='2')
    
plt.show()

AttributeError: 'TaxiEnv' object has no attribute '_get_obs'

<Figure size 432x288 with 0 Axes>

In [15]:
plt.figure()
for i in range(n_episodes):
    x = np.asarray(q_estimation[i])
    plt.plot(x[:,1])

plt.show()

NameError: name 'q_estimation' is not defined

<Figure size 432x288 with 0 Axes>

## Testing

In [59]:
agent.temperature = 10
for _ in range(1000):
    environment.exploration_episode(render=False)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+
  (East)
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : 

In [60]:
for _ in range(5):
    environment.evaluation_episode(render=False)